<a href="https://colab.research.google.com/github/rakezhjab/Advanced-machine-learning-assessment/blob/main/A2C_SEED70_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Mario game setup**

In [17]:
!pip install gym_super_mario_bros==7.3.0 nes_py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
# Importing the game
import gym_super_mario_bros
# Importing the Joypad wrapper
from nes_py.wrappers import JoypadSpace
# Importing the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [19]:
# Setup game
env = gym_super_mario_bros.make('SuperMarioBros2-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

**Preprocessing the environment**

In [20]:
# Installing pytorch
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
tcmalloc: large alloc 1821458432 bytes == 0x306a000 @  0x7fac20c0a1e7 0x4a3940 0x5b438c 0x5ea94f 0x5939cb 0x594cd3 0x5d0ecb 0x59aeca 0x515655 0x593dd7 0x511e2c 0x593dd7 0x511e2c 0x549e0e 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549e0e 0x4bca8a 0x59c019
tcmalloc: large alloc 1821458432 bytes == 0x6f97e000 @  0x7fac20c0a1e7 0x4a3940 0x5b438c 0x64cfe7 0x59b076 0x515655 0x593dd7 0x511e2c 0x593dd7 0x511e2c 0x549e0e 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549e0e 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549576
tcmalloc: large alloc 1821458432 bytes == 0xdc292000 @  0x7fac20c0a1e7 0x4a3940 0x59b5e2 0x63a515 0x63bd66 0x63be16 0x59afff 0x515655 0x593dd7 0x

In [21]:
# Installing the stable baselines for Reinforcement Learning stuff
!pip install stable-baselines3[extra]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-1.12.0-cp37-cp37m-manylinux1_x86_64.whl (776.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.10.1+cu113
    Uninstalling torch-1.10.1+cu113:
      Successfully uninstalled torch-1.10.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.2+cu113 requires torch==1.10.1, but you have torch 1.12.0 which is incompatible.
torchaudio 0.10.1+cu113 requires torch==1.10.1, but you have torch 1.12.0 which is incompatible.


In [22]:
# Importing the Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation
# Importing the Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Importing the Matplotlib to show the impact of frame stacking
from matplotlib import pyplot as plt

**Train the Reinforcement Learning model**

In [23]:
# Import os for file path management
import os 
# Import PPO for algos
from stable_baselines3 import A2C
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [24]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [25]:
CHECKPOINT_DIRECTORY = './training/'
LOG_DIRECTORY = './logs/'

In [26]:
# Setup the model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIRECTORY)

In [27]:
# AI model started
model = A2C('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIRECTORY, learning_rate=0.000001, seed=70,
            n_steps=512) 

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [28]:
# Train the AI model, now AI model starts learning
model.learn(total_timesteps=1000000, callback=callback)

Logging to ./logs/A2C_2


/usr/local/lib/python3.7/dist-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.23e+04 |
|    ep_rew_mean        | 347      |
| time/                 |          |
|    fps                | 185      |
|    iterations         | 100      |
|    time_elapsed       | 275      |
|    total_timesteps    | 51200    |
| train/                |          |
|    entropy_loss       | -1.95    |
|    explained_variance | -0.0804  |
|    learning_rate      | 1e-06    |
|    n_updates          | 99       |
|    policy_loss        | -0.473   |
|    value_loss         | 0.34     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 1.46e+04 |
|    ep_rew_mean        | 357      |
| time/                 |          |
|    fps                | 187      |
|    iterations         | 200      |
|    time_elapsed       | 547      |
|    total_timesteps    | 102400   |
| train/                |          |
|

In [29]:
model.save('A2CSeedTestmodel')

**Testing and Loading the Model**

In [30]:
# Loading the model
model = A2C.load('./training/best_model_1000000')